# Train a Neural Network for Classification

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


# Load Titanic dataset
# df = pd.read_csv("train.csv")
url = 'https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv'
df = pd.read_csv(url)

# Select relevant features
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
df = df[features + ["Survived"]].dropna()

# Convert categorical to numerical
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])

# Define inputs & outputs
X = df[features].values
y = df["Survived"].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.float32).view(-1, 1), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the Neural Network
class TitanicNN(nn.Module):
    def __init__(self):
        super(TitanicNN, self).__init__()
        self.fc1 = nn.Linear(6, 10)  # Input: 6 features, Hidden: 10 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 1)  # Output: 1 neuron
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for binary classification

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)  # Ensures output is between 0-1
        return x

# Define Loss Function & Optimizer
model = TitanicNN()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the Model
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# Evaluate the Model
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred > 0.5).float()  # Convert probabilities to 0 or 1
    accuracy = (y_pred == y_test).sum().item() / y_test.shape[0]

print(f"Model Accuracy: {accuracy:.2%}")


Epoch 0, Loss: 0.6441
Epoch 100, Loss: 0.2789
Epoch 200, Loss: 0.2354
Epoch 300, Loss: 0.2142
Epoch 400, Loss: 0.1846
Model Accuracy: 76.92%


# Model Training and Evaluation Process

**Preprocessing Steps:**

* ✔️ Removed missing values.
* ✔️ Converted Sex (Male/Female) to 0/1.
* ✔️ Normalized numerical features.
* ✔️ Split data into train (80%) & test (20%).

**Key Components:**

* ✔️ `ReLU()` → Helps in feature extraction.
* ✔️ `Sigmoid()` → Converts output to probability (0-1).

**Loss Function:**

* ✅ `BCELoss()` is used instead of `MSE` since it works better for classification.

**Training Process:**

1.  **Forward Pass (Compute predictions):**
    * The model takes the input data and generates predictions.
2.  **Compute Loss (Compare predictions with actual values):**
    * The `BCELoss` function calculates the difference between the predicted probabilities and the actual target values.
3.  **Backpropagation (Update weights):**
    * The model adjusts its internal parameters (weights) based on the calculated loss to improve future predictions.

**Evaluation Process:**

* ✔️ Converts probabilities to 0/1 based on a 0.5 threshold.
* ✔️ Compares predictions with actual values.
* ✔️ Computes the Accuracy of the model.